In [3]:
# install firecrawl if not installed 
%pip install firecrawl

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0

In [1]:
FIRECRAWL_API_KEY = 'fc-ce38ce36fb854e9197959647181a8163'

In [4]:
from firecrawl import FirecrawlApp

# Initialize the FirecrawlApp with your API key
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

# Scrape the Al Arabiya Middle East News page
scrape_result = app.scrape_url(
    url="https://english.alarabiya.net/News/middle-east",
    formats=["markdown", "html"]
)

# Extract articles from the scrape result
articles = scrape_result.articles if hasattr(scrape_result, 'articles') else []
print(f"Found {len(articles)} articles.")

# Save the results to a file
with open('arab_dataset.csv', 'w', encoding='utf-8') as file:
    # Write the header
    file.write("Title,Link,Date,Content\n")
    
    # Write the data
    for article in articles:
        title = article.get('title', '').replace(',', ' ')  # Remove commas from title
        link = article.get('link', '')
        date = article.get('date', '')
        content = article.get('content', '').replace(',', ' ')  # Remove commas from content
        file.write(f"{title},{link},{date},{content}\n")

Found 0 articles.
